# **Initialization**

In [1]:
from keras.applications import ResNet50
from keras.layers import Input, Conv3D, MaxPooling3D, Flatten, Dense, GlobalAveragePooling3D, add, BatchNormalization, Activation, Dropout
from keras.models import Model
import keras
from google.colab import drive
import scipy.io
from keras.regularizers import l2
from keras import backend as K
import tensorflow
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
import numpy as np
import cv2
import pandas as pd

# **Load MRI Data from Matlab file (run once if data didn't change)**

In [ ]:
# Read in Matlab data
drive.mount('/content/drive')

matlab_datasets = []

for i in range(1, 17):
  try:
    mat = scipy.io.loadmat('/content/drive/MyDrive/Fall 2023/EECS E6893 - Big Data Analytics/Final_Project/MRI Matlab Data/image_data{num}.mat'.format(num = i))
    matlab_datasets.append(mat)
  except:
    continue

print(len(matlab_datasets))

Mounted at /content/drive
15


In [ ]:
patient_ids = []
designators = []
imaging_types = []
techniques = []
datasets = []

# create a Pandas dataframe
for matfile in matlab_datasets:

  try:
    for line in matfile['image_data'][0]:
      patient_ids.append(line[0][0])
      designators.append(line[1][0])
      imaging_types.append(line[2][0])
      techniques.append(line[3][0])
      datasets.append(line[4])
  except:
    print('line issue: ', line[5][0])

data = {
    "Patient_ID": patient_ids,
    "Designator": designators,
    "Imaging_Type": imaging_types,
    "Technique": techniques,
    "Data": datasets,
}

df = pd.DataFrame(data)
print(df)

# save Pandas dataframe
df.to_pickle('/content/drive/MyDrive/Fall 2023/EECS E6893 - Big Data Analytics/Final_Project/MRI Matlab Data/mri_data_pandas.pkl')

# print(df.iloc[0]['Data'].shape)


# **Load MRI Data from Pandas DF (start here)**

In [2]:
# load Pandas dataframe
drive.mount('/content/drive')
file_name = '/content/drive/MyDrive/Fall 2023/EECS E6893 - Big Data Analytics/Final_Project/MRI Matlab Data/mri_data_pandas.pkl'
df_loaded = pd.read_pickle(file_name)
print(df_loaded)


Mounted at /content/drive
      Patient_ID    Designator             Imaging_Type  \
0       00123691    responsive                      MRA   
1       00123691    responsive                      MRA   
2       00123691    responsive                      MRA   
3       00123691    responsive                      MRA   
4       00123691    responsive                      MRA   
...          ...           ...                      ...   
45652   15871226  unresponsive  Brain MRI , Stroke (-C)   
45653   15871226  unresponsive  Brain MRI , Stroke (-C)   
45654   15871226  unresponsive  Brain MRI , Stroke (-C)   
45655   15871226  unresponsive  Brain MRI , Stroke (-C)   
45656   15871226  unresponsive  Brain MRI , Stroke (-C)   

                                               Technique  \
0      MR.1.2.840.113619.2.80.3709905447.15220.168109...   
1      MR.1.2.840.113619.2.80.3709905447.15220.168109...   
2      MR.1.2.840.113619.2.80.3709905447.15220.168109...   
3      MR.1.2.840.113619.

# **Organize Data for Training**

In [3]:
def get_patient_list(mat_file):
  """
  Input: mat_file - matlab file
  Output: patient_list - set of patient IDs
  """
  patient_list = []
  for line in mat_file['image_data'][0]:
    patient_list.append(line[0][0])

  patient_list = set(patient_list)

  return patient_list


def map_patient_to_img_technique(mat_file, patient_list):
  """
  Input: mat_file - matlab file
         patient_list - set or list of patients
  Output: patient_img_technique_map - dict of patients with their MRI image type
  """
  patient_img_technique_map = {}

  for patient_id in patient_list:
    img_techniques = []
    for line in mat_file['image_data'][0]:
      if line[0][0] == patient_id and line[3][0] not in img_techniques:
        img_techniques.append(line[3][0])
    patient_img_technique_map[patient_id] = img_techniques

  return patient_img_technique_map


def rescale_img(img_array, resolution):
  """
  Input: img_array - 2D numpy image array
         resolution - tuple of (height, width)
  Output: res - rescaled 2D numpy image array
  """
  if img_array.shape != resolution:
    res = cv2.resize(img_array, dsize=resolution, interpolation=cv2.INTER_CUBIC)
  else:
    res = img_array
  return res


def stack_mri_slices(img_slices):
  """

  """
  img_3D = np.dstack(img_slices)
  img_3D = img_3D[:, :, :, np.newaxis]
  return img_3D

# print(df_loaded['Technique'].value_counts())


In [4]:
# rescale images to desired resolution
TARGET_RESOLUTION = (256, 256)
for idx in df_loaded.index:
  try:
    df_loaded['Data'][idx] = rescale_img(df_loaded['Data'][idx], TARGET_RESOLUTION)
    print(df_loaded['Data'][idx].shape)
  except:
    print('failed at: ', idx)

Streaming output truncated to the last 5000 lines.
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(25

In [5]:
# get list of patients and shuffle

patient_list = df_loaded['Patient_ID'].unique() # get a list of patients
print(patient_list)
patient_list = np.delete(patient_list, 58)      # remove a patient ID
patient_list = np.delete(patient_list, np.where(patient_list == '02445263'))
patient_list = np.delete(patient_list, np.where(patient_list == '15816944'))
print(len(patient_list))
print(patient_list)
np.random.shuffle(patient_list)                 # shuffle the patient list
# print(patient_list)

train_patients = patient_list[0:46]   # 70% of patients for training
# print(train_patients)
test_patients = patient_list[46:]     # 30% of patients for testing
# print(test_patients)

['00123691' '00123703' '00344067' '00587752' '00659695' '00717803'
 '00731674' '00748650' '00776976' '00788250' '00969116' '01248465'
 '01633823' '01998579' '02028394' '02105211' '02214560' '02445263'
 '03162787' '03177895' '03254239' '03439818' '03482142' '03527513'
 '03621305' '03787218' '04046509' '04330381' '04447605' '05139693'
 '09713909' '09837951' '10488810' '11245058' '12223695' '12245385'
 '12663851' '13195492' '14974791' '15062060' '15388959' '15399737'
 '15699459' '15722276' '15726562' '15744922' '15751694' '15790899'
 '15816944' '15823324' '15831031' '15837723' '15843573' '15902311'
 '15928409' '15955689' '01134548' '02295244' '02982450' '03024795'
 '03806603' '04754369' '09185913' '13431080' '14763045' '15530925'
 '15631310' '15705952' '15871226']
66
['00123691' '00123703' '00344067' '00587752' '00659695' '00717803'
 '00731674' '00748650' '00776976' '00788250' '00969116' '01248465'
 '01633823' '01998579' '02028394' '02105211' '02214560' '03162787'
 '03177895' '03254239' '

In [ ]:

df_loaded.loc[df_loaded['Patient_ID'] == '15816944', 'Technique']



In [6]:
# build training/testing dataset

x_data_train_dict = {}
y_data_train_dict = {}
x_data_test_dict = {}
y_data_test_dict = {}
x_data_all_dict = {}
y_data_all_dict = {}

num_slices = 64

# TRAINING data x
for patient_id in train_patients:
  try:
    patient_df = df_loaded.loc[df_loaded['Patient_ID'] == patient_id]
    patient_mri_data = patient_df.loc[patient_df['Technique'] == 'SWAN']
    # print(patient_mri_data)

    patient_mri_data = stack_mri_slices(patient_mri_data['Data'].to_numpy()[0:num_slices])
    # print(patient_mri_data.shape)
    x_data_train_dict[patient_id] = patient_mri_data

  except:
    print('no data: ', patient_id)
    patient_df = df_loaded.loc[df_loaded['Patient_ID'] == patient_id]
    patient_mri_data = patient_df.loc[patient_df['Technique'] == 'Ax DWI Asset']
    # print(patient_mri_data)

    patient_mri_data = stack_mri_slices(patient_mri_data['Data'].to_numpy()[0:num_slices])
    # print(patient_mri_data.shape)
    x_data_train_dict[patient_id] = patient_mri_data

# print(len(x_data_train_dict))
# print(x_data_train_dict)

# TRAINING data y
for patient_id in train_patients:
  outcome = df_loaded.loc[df_loaded['Patient_ID'] == patient_id, 'Designator'].iloc[0]
  if outcome == 'responsive':
    y_data_train_dict[patient_id] = 1
  elif outcome == 'unresponsive':
    y_data_train_dict[patient_id] = 0

print(len(y_data_train_dict))
# print(y_data_train_dict)

# TESTING data x
# print(len(test_patients))
for patient_id in test_patients:
  # print(patient_id)
  try:
    patient_df = df_loaded.loc[df_loaded['Patient_ID'] == patient_id]
    patient_mri_data = patient_df.loc[patient_df['Technique'] == 'SWAN']
    # print(patient_mri_data)

    patient_mri_data = stack_mri_slices(patient_mri_data['Data'].to_numpy()[0:num_slices])

    # print(patient_mri_data.shape)
    x_data_test_dict[patient_id] = patient_mri_data

  except:
    # print('no data: ', patient_id)
    patient_df = df_loaded.loc[df_loaded['Patient_ID'] == patient_id]
    patient_mri_data = patient_df.loc[patient_df['Technique'] == 'Ax DWI Asset']
    # print(patient_mri_data)

    patient_mri_data = stack_mri_slices(patient_mri_data['Data'].to_numpy()[0:num_slices])
    # print(patient_mri_data.shape)
    x_data_test_dict[patient_id] = patient_mri_data

print(len(x_data_test_dict))
# print(x_data_test_dict)


# TESTING data y
for patient_id in test_patients:
  outcome = df_loaded.loc[df_loaded['Patient_ID'] == patient_id, 'Designator'].iloc[0]
  if outcome == 'responsive':
    y_data_test_dict[patient_id] = 1
  elif outcome == 'unresponsive':
    y_data_test_dict[patient_id] = 0

print(len(y_data_test_dict))
# print(y_data_train_dict)

# print(patient_mri_data.shape)
# print(patient_df['Technique'].value_counts())



46
20
20


In [7]:
print('x data train shape:')
for patient in x_data_train_dict:
  print(x_data_train_dict[patient].shape)

print('x data test shape:')
for patient in x_data_test_dict:
  print(x_data_test_dict[patient].shape)

x data train shape:
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
x data test shape:
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 256, 64, 1)
(256, 2

In [8]:
x_train = []
y_train = []
x_test = []
y_test = []

for patient_id in train_patients:
  x_train.append(x_data_train_dict[patient_id])
  y_train.append(y_data_train_dict[patient_id])

for patient_id in test_patients:
  x_test.append(x_data_test_dict[patient_id])
  y_test.append(y_data_test_dict[patient_id])

x_train = np.asarray(x_train)
print(x_train.shape)
y_train = np.asarray(y_train)
print(y_train.shape)
x_test = np.asarray(x_test)
print(x_test.shape)
y_test = np.asarray(y_test)
print(y_test.shape)

train_loader = tf.data.Dataset.from_tensor_slices((x_train, y_train))

batch_size = 1
# Augment the on the fly during training.
train_dataset = (
    train_loader.shuffle(len(x_train))
    # .map(train_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)

test_loader = tf.data.Dataset.from_tensor_slices((x_test, y_test))

batch_size = 1
# Augment the on the fly during training.
test_dataset = (
    test_loader.shuffle(len(x_test))
    # .map(train_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)

(46, 256, 256, 64, 1)
(46,)
(20, 256, 256, 64, 1)
(20,)


# **3D Resnet sample #1**

In [ ]:
# Define input shape for 3D data
depth = 100
height = 256
width = 256
channels = 3
input_shape = (depth, height, width, channels)
input_tensor = Input(shape=input_shape)


# Load ResNet-50 without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

# Add 3D convolutional layers on top of the base model
x = Conv3D(64, (3, 3, 3), activation='relu')(base_model.output)
x = MaxPooling3D(pool_size=(2, 2, 2))(x)

# Flatten the output and add fully connected layers
x = GlobalAveragePooling3D()(x)
x = Dense(512, activation='relu')(x)
num_classes = 2
output = Dense(num_classes, activation='softmax')(x)

# Create the new model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model with an appropriate optimizer, loss function, and metrics
model.compile(optimizer='adam', loss='categorical_crossentropy')



# **3D Resnet sample #2**

In [ ]:
depth = 128
height = 256
width = 256
channels = 1
input_shape = (depth, height, width, channels)
# input_shape = (depth, height, width)
input_shape = (height, width, depth, channels)

kernel_regularizer=l2(1e-4)

C1, C2 = tf.constant(70, dtype='float32'), tf.constant(1000, dtype="float32")

def score(y_true, y_pred):
    tf.dtypes.cast(y_true, tf.float32)
    tf.dtypes.cast(y_pred, tf.float32)
    sigma = y_pred[:, 2] - y_pred[:, 0]
    fvc_pred = y_pred[:, 1]

    #sigma_clip = sigma + C1
    sigma_clip = tf.maximum(sigma, C1)
    delta = tf.abs(y_true[:, 0] - fvc_pred)
    delta = tf.minimum(delta, C2)
    sq2 = tf.sqrt( tf.dtypes.cast(2, dtype=tf.float32) )
    metric = (delta / sigma_clip)*sq2 + tf.math.log(sigma_clip* sq2)
    return K.mean(metric)

def qloss(y_true, y_pred):
    tf.dtypes.cast(y_true, tf.float32)
    tf.dtypes.cast(y_pred, tf.float32)
    # Pinball loss for multiple quantiles
    qs = [0.2, 0.50, 0.8]
    q = tf.constant(np.array([qs]), dtype=tf.float32)
    e = y_true - y_pred
    v = tf.maximum(q*e, (q-1)*e)
    return K.mean(v)

def mloss(_lambda):
    def loss(y_true, y_pred):
        return _lambda * qloss(y_true, y_pred) + (1 - _lambda)*score(y_true, y_pred)
    return loss

def resnet_3d():
    img_input = Input(shape=input_shape, name="img_input")

    ## First layer
    conv1 = Conv3D(filters=32, kernel_size=(5, 5, 5),
                           strides=(2,2,2), padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(img_input)
    #print(conv1.shape)
    conv11 = Conv3D(filters=32, kernel_size=(5, 5, 5),
                           strides=(1,1,1), padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(conv1)

    norm1 = BatchNormalization(axis=-1)(conv11)
    relu1 = Activation("relu")(norm1)
    #print(relu1.shape)
    residual1 = Conv3D(filters=32, kernel_size=(3, 3, 3),
                           strides=(1,1,1), padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(relu1)
    #print(residual1.shape)
    resblock1 = add([conv1, residual1])

    conv2 = Conv3D(filters=64, kernel_size=(5, 5, 5),
                           strides=(2,2,2), padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(resblock1)

    conv22 = Conv3D(filters=64, kernel_size=(5, 5, 5),
                           strides=(1,1,1), padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(conv2)


    norm2 = BatchNormalization(axis=-1)(conv22)
    relu2 = Activation("relu")(norm2)
    #print(relu1.shape)
    residual2 = Conv3D(filters=64, kernel_size=(3, 3, 3),
                           strides=(1,1,1), padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(relu2)
    #print(residual1.shape)
    resblock2 = add([conv2, residual2])


    conv3 = Conv3D(filters=64, kernel_size=(3, 3, 3),
                           strides=(2,2,2), padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(resblock2)

    conv33 = Conv3D(filters=128, kernel_size=(3, 3, 3),
                           strides=(1,1,1), padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(conv3)


    norm3 = BatchNormalization(axis=-1)(conv3)
    relu3 = Activation("relu")(norm3)
    #print(relu1.shape)
    residual3 = Conv3D(filters=64, kernel_size=(3, 3, 3),
                           strides=(1,1,1), padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(relu3)
    #print(residual1.shape)
    resblock3 = add([conv3, residual3])

    conv4 = Conv3D(filters=16, kernel_size=(3, 3, 3),
                           strides=(2,2,2), padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(resblock3)

    conv44 = Conv3D(filters=16, kernel_size=(3, 3, 3),
                           strides=(1,1,1), padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(conv4)


    norm4 = BatchNormalization(axis=-1)(conv44)
    relu4 = Activation("relu")(norm4)
    #print(relu1.shape)
    residual4 = Conv3D(filters=16, kernel_size=(3, 3, 3),
                           strides=(1,1,1), padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(relu4)
    #print(residual1.shape)
    resblock4 = add([conv4, residual4])

    conv5 = Conv3D(filters=16, kernel_size=(3, 3, 3),
                           strides=(2,2,1), padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(resblock4)


#     #cnn = Conv3D(32, kernel_size=(5), strides=(2), padding='same', activation='relu', kernel_initializer='he_uniform', name="conv_3d_1")(img_input)
#     cnn = MaxPooling3D(pool_size=(2,2,2), strides=(2))(cnn)
#     cnn = Conv3D(64, kernel_size=(3), strides=(2), padding='same', activation='relu', kernel_initializer='he_uniform', name="conv_3d_2")(img_input)
#     cnn = MaxPooling3D(pool_size=(2,2,2), strides=(2, 2, 2))(cnn)
    flatten = Flatten()(conv5)
    final_cnn_dense = Dense(512, activation='relu')(flatten)
    model = Model(
        inputs=[img_input],
        outputs=[final_cnn_dense],
    )
    return model


def create_model():
#     cnn_model = create_cnn_model()

    cnn_model = resnet_3d()

    x = Dropout(0.5)(cnn_model.output)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.4)(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.4)(x)
    # x = Flatten()(x)
    x = Dense(1, activation="sigmoid", name="final_dense")(x)

    model = Model(inputs=[cnn_model.input], outputs=x)
    adam = tensorflow.keras.optimizers.Adam(lr=0.004)

    # radam = tfa.optimizers.RectifiedAdam(lr=5e-3,total_steps=10000, warmup_proportion=0.1,min_lr=1e-5)
    # ranger = tfa.optimizers.Lookahead(radam, sync_period=6, slow_step_size=0.5)

    # model.compile(loss=mloss(0.8), optimizer="adam", metrics=[score])
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=[score])

    return model

m22 = create_model()
# m22.summary()

m22.fit(train_dataset)


# **3D Resnet sample #3**

In [ ]:
# Change the input layer to accept 3D data.


depth = 32
height = 256
width = 256
channels = 2
input_shape = (depth, height, width, channels)
input_tensor = Input(shape=input_shape)

# Update the first convolutional layer to be 3D.
from keras.layers import Conv3D
filters = 64
kernel_size = (3, 3, 3)
x = Conv3D(filters, kernel_size, strides=(1, 1, 1), padding='same', activation='relu')(input_tensor)

# Modify pooling layers to 3D pooling.
from keras.layers import MaxPooling3D

x = MaxPooling3D(pool_size=(2, 2, 2))(x)

# ResNet-50 consists of residual blocks. Modify the convolutional layers in these blocks to handle 3D input.
from keras.layers import Conv3D, BatchNormalization, Activation, Add

def residual_block_3d(x, filters, kernel_size=(3, 3, 3), strides=(1, 1, 1)):
    # Shortcut
    shortcut = x

    # First convolution layer
    x = Conv3D(filters, kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Second convolution layer
    x = Conv3D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)

    # Adding the shortcut to the main path
    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

x = residual_block_3d(x, 64)

# Adjust the Fully Connected (Dense) Layers
from keras.layers import Dense, Flatten

x = Flatten()(x)
units = 2
output_tensor = Dense(units, activation='relu')(x)

# Create the Model
from keras.models import Model

model = Model(inputs=input_tensor, outputs=output_tensor)

# Compile and Train
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.fit(...)  # Add your training data here
print('test')

test


# 3D Resnet sample #4

In [ ]:
def get_model(width=128, height=128, depth=64):
    """Build a 3D convolutional neural network model."""

    inputs = Input((width, height, depth, 1))

    x = Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = MaxPooling3D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = MaxPooling3D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = MaxPooling3D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = MaxPooling3D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = GlobalAveragePooling3D()(x)
    x = Dense(units=512, activation="relu")(x)
    x = Dropout(0.3)(x)

    outputs = Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = Model(inputs, outputs, name="3dcnn")
    return model


# Build model.
WIDTH = 256
HEIGHT = 256
DEPTH = 64
model = get_model(width=WIDTH, height=HEIGHT, depth=DEPTH)
model.summary()

initial_learning_rate = 0.0001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=["acc"],
)

train_out = model.fit(train_dataset, validation_data=test_dataset, epochs=50)

In [ ]:
# Save model
model.save("/content/drive/MyDrive/Fall 2023/EECS E6893 - Big Data Analytics/Final_Project/MRI Matlab Data/MRI_Model.keras")

In [ ]:
import matplotlib.pyplot as plt

print(train_out.history.keys())

# summarize history for accuracy
plt.plot(train_out.history['acc'])
plt.plot(train_out.history['val_acc'])
plt.ylim([0, 1])
plt.title('MRI model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(train_out.history['loss'])
plt.plot(train_out.history['val_loss'])
plt.ylim([0, 1])
plt.title('MRI model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [9]:
# Load model
loaded_model = keras.saving.load_model("/content/drive/MyDrive/Fall 2023/EECS E6893 - Big Data Analytics/Final_Project/MRI Matlab Data/MRI_Model.keras")

In [ ]:
model.predict(test_dataset.take(4))

In [79]:
x_data_all_dict = {**x_data_train_dict, **x_data_test_dict}
y_data_all_dict = {**y_data_train_dict, **y_data_test_dict}

print('label:', y_data_all_dict['00123691'])

# for stack in test_dataset:
  # print(stack[1].numpy())

eval_data = x_data_all_dict['00123691']
eval_data = eval_data[None, :]
print(eval_data.shape)
eval_loader = tf.data.Dataset.from_tensors(eval_data)

loaded_model.predict(eval_loader)



label: 1
(1, 256, 256, 64, 1)
1/1 [==============================] - 0s 13ms/step


array([[0.49848634]], dtype=float32)